In [1]:
# загружаем основные библиотеки для преобразования
import sys

import pandas as pd
import itertools
import nltk
import numpy as np
import pandas as pd
import re
from multiprocessing import Pool
from nltk.corpus import stopwords
from pymystem3 import Mystem

from nltk.stem import SnowballStemmer
import pymorphy2

skip_words = []

# для нормализации цены 
from math import log

In [2]:
# загружаем нашу тренировочную выборку, удяляем дубликаты по описанию, и смотрим что у нас там вообще есть
df = pd.read_csv('train.csv')
print(len(df))
df = df.drop_duplicates(subset='description')
print(len(df))
print(df.head())
print(df.shape)

489517
482015
   item_id                                              title  \
0        0                                            Картина   
1        1                        Стулья из прессованной кожи   
2        2                                 Домашняя мини баня   
3        3  Эксклюзивная коллекция книг "Трансаэро" + подарок   
4        4                                       Ноутбук aser   

                                         description    price  category_id  
0                         Гобелен. Размеры 139х84см.   1000.0           19  
1  Продам недорого 4 стула из светлой прессованно...   1250.0           22  
2  Мини баня МБ-1(мини сауна), предназначена для ...  13000.0           37  
3  Продам эксклюзивную коллекцию книг, выпущенную...   4000.0           43  
4  Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...  19000.0            1  
(482015, 5)


In [3]:
# смотрим что есть у нас в поле category_id

tags = sorted(set(df.category_id.values))
print(tags)

#категории числовые так что преобразование в числа пропускаем

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]


In [4]:
example = df.loc[2, 'description']
print(example)

Мини баня МБ-1(мини сауна), предназначена для принятия тепловых процедур в бытовых условиях(дома, на даче), а также в спортивных, оздоровительных, косметических, лечебных организациях и учреждениях. Она оказывает общеукрепляющее и профилактическое действие на организм человека. Номинальное напряжение переменного тока 220 В Максимальная мощность 2000 Вт Максимальная температура нагретого воздуха в термочехле (градусов С)	90 – 100 Время достижения рабочей температуры 1,5 мин. Время достижения максимальной температуры 10 мин. Габаритные размеры тепловой камеры в рабочем состоянии, мм не более: - Длина 950 - Ширина 900 - Высота 1100 Масса 6,3 - 7,3 кг Мини баня станет отличным подарком как для мужчин, так и для женщин.


In [5]:
stemmer = SnowballStemmer('russian')
lemmer = Mystem()

Installing mystem to /home/jovyan/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz


In [6]:
def preproc1(text):
    text = re.sub(u"[^a-zа-я0-9]", " ", text.lower())
    text = re.sub(u"\s+", " ", text)
    tokens = lemmer.lemmatize(text)
    if len(tokens) == 0:
        print('None')
        return 'none'
    return " ".join([i.strip() for i in tokens if i.strip()])

print(preproc1(example))

мини баня мб 1 мини сауна предназначать для принятие тепловой процедура в бытовой условие дом на дача а также в спортивный оздоровительный косметический лечебный организация и учреждение она оказывать общеукрепляющий и профилактический действие на организм человек номинальный напряжение переменный ток 220 в максимальный мощность 2000 вт максимальный температура нагревать воздух в термочехол градус с 90 100 время достижение рабочий температура 1 5 мина время достижение максимальный температура 10 мина габаритный размер тепловой камера в рабочий состояние мм не более длина 950 ширина 900 высота 1100 масса 6 3 7 3 кг мини баня становиться отличный подарок как для мужчина так и для женщина


In [7]:
def preproc2(text):
    text = re.sub(u"[^a-zа-я0-9]", " ", text.lower())
    text = re.sub(u"\s+", " ", text)
    tokens = lemmer.lemmatize(text)
    tmp = []
    for lem, res in zip(tokens, lemmer.analyze(text)):
        lem = lem.strip()
        if lem:
            if "analysis" in res:
                try:
                    res = res["analysis"][0]["gr"].split(",")[0].split("=")[0]
                except:
                    res = "UNKN"
            else:
                res = "UNKN"
            lem = "{}_{}".format(lem, res)
            tmp.append(lem)
    return " ".join(tmp)

print(preproc2(example))

мини_COM баня_S мб_S 1_UNKN мини_COM сауна_S предназначать_V для_PR принятие_S тепловой_A процедура_S в_PR бытовой_A условие_S дом_S на_PR дача_S а_CONJ также_ADV в_PR спортивный_A оздоровительный_A косметический_A лечебный_A организация_S и_CONJ учреждение_S она_SPRO оказывать_V общеукрепляющий_A и_CONJ профилактический_A действие_S на_PR организм_S человек_S номинальный_A напряжение_S переменный_A ток_S 220_UNKN в_PR максимальный_A мощность_S 2000_UNKN вт_S максимальный_A температура_S нагревать_V воздух_S в_PR термочехол_S градус_S с_PR 90_UNKN 100_UNKN время_S достижение_S рабочий_A температура_S 1_UNKN 5_UNKN мина_S время_S достижение_S максимальный_A температура_S 10_UNKN мина_S габаритный_A размер_S тепловой_A камера_S в_PR рабочий_A состояние_S мм_S не_PART более_ADV длина_S 950_UNKN ширина_S 900_UNKN высота_S 1100_UNKN масса_S 6_UNKN 3_UNKN 7_UNKN 3_UNKN кг_S мини_COM баня_S становиться_V отличный_A подарок_S как_CONJ для_PR мужчина_S так_ADVPRO и_CONJ для_PR женщина_S


In [8]:
def preproc3(text):
    tokens = []
    text = re.sub(u"[^a-zа-я0-9]", " ", text.lower())
    text = re.sub(u"\s+", " ", text)
    for token in re.split(' ', text.lower()):
        if len(token) > 1:
            norm_token = stemmer.stem(token)
            if norm_token in skip_words:
                continue
            tokens.append(norm_token)
    if len(tokens) == 0:
        return 'None'
    return ' '.join(tokens)

print(preproc3(example))

мин бан мб мин саун предназнач для принят теплов процедур бытов услов дом на дач такж спортивн оздоровительн косметическ лечебн организац учрежден он оказыва общеукрепля профилактическ действ на организм человек номинальн напряжен перемен ток 220 максимальн мощност 2000 вт максимальн температур нагрет воздух термочехл градус 90 100 врем достижен рабоч температур мин врем достижен максимальн температур 10 мин габаритн размер теплов камер рабоч состоян мм не бол длин 950 ширин 900 высот 1100 масс кг мин бан станет отличн подарк как для мужчин так для женщин


In [9]:
df['preproc1_des'] = [preproc1(el) for el in df['description']]
df['preproc1_tit'] = [preproc1(el) for el in df['title']]

In [10]:
df['preproc2_des'] = [preproc2(el) for el in df['description']]
df['preproc2_tit'] = [preproc2(el) for el in df['title']]

In [11]:
df['preproc3_des'] = [preproc3(el) for el in df['description']]
df['preproc3_tit'] = [preproc3(el) for el in df['title']]

In [12]:
# разбиваем цену на 14 диапазонов по %% и берем в рамках группы log по основанию line_max
for min_prc, max_prc in zip([0, 1, 10, 20, 30, 40, 45, 50, 55, 60, 70, 80, 90, 99], [1, 10, 20, 30, 40, 45, 50, 55, 60, 70, 80, 90, 99, 100]):
    line_min = np.percentile(df['price'], min_prc)
    line_max = np.percentile(df['price'], max_prc)
    df['price_' + str(max_prc) + '_log_nrm'] = [log(el,  line_max) if line_min < el <= line_max else 0 for el in df['price']]
    
df.isnull().any()

item_id              False
title                False
description          False
price                False
category_id          False
preproc1_des         False
preproc1_tit         False
preproc2_des         False
preproc2_tit         False
preproc3_des         False
preproc3_tit         False
price_1_log_nrm      False
price_10_log_nrm     False
price_20_log_nrm     False
price_30_log_nrm     False
price_40_log_nrm     False
price_45_log_nrm     False
price_50_log_nrm     False
price_55_log_nrm     False
price_60_log_nrm     False
price_70_log_nrm     False
price_80_log_nrm     False
price_90_log_nrm     False
price_99_log_nrm     False
price_100_log_nrm    False
dtype: bool

In [13]:
# сохраняем полученное (977Mb)
df.to_csv('prep_dataset.csv', encoding='utf-8')

In [14]:
# тоже самое для test файла
tf = pd.read_csv('test.csv')
print(len(tf))
print(tf.head())
print(tf.shape)

tf['preproc1_des'] = [preproc1(el) for el in tf['description']]
tf['preproc1_tit'] = [preproc1(el) for el in tf['title']]
tf['preproc2_des'] = [preproc2(el) for el in tf['description']]
tf['preproc2_tit'] = [preproc2(el) for el in tf['title']]
tf['preproc3_des'] = [preproc2(el) for el in tf['description']]
tf['preproc3_tit'] = [preproc2(el) for el in tf['title']]

# разбиваем цену на 14 диапазонов по %% (на основе обучающей выборки) и берем в рамках группы log по основанию line_max
for min_prc, max_prc in zip([0, 1, 10, 20, 30, 40, 45, 50, 55, 60, 70, 80, 90, 99], [1, 10, 20, 30, 40, 45, 50, 55, 60, 70, 80, 90, 99, 100]):
    line_min = np.percentile(df['price'], min_prc)
    line_max = np.percentile(df['price'], max_prc)
    tf['price_' + str(max_prc) + '_log_nrm'] = [log(el,  line_max) if line_min < el <= line_max else 0 for el in tf['price']]
    
tf.isnull().any()

# сохраняем полученное (540Mb)
tf.to_csv('prep_test.csv', encoding='utf-8')

243166
   item_id                     title  \
0   489517    Стоик журнальный сталь   
1   489518             iPhone 5 64Gb   
2   489519                Утеплитель   
3   489520       Пальто демисезонное   
4   489521  Samsung syncmaster T200N   

                                         description    price  
0  продам журнальный столик изготавливаю столы из...  10000.0  
1  Телефон в хорошем состоянии. Комплект, гаранти...  12500.0  
2  ТЕПЛОПЕЛЕН-ЛИДЕР ТЕПЛА!!! Толщина утеплителя :...    250.0  
3  Продам пальто женское (букле) в отличном состо...   1700.0  
4  Условно рабочий, проблема в панели настройки м...   1000.0  
(243166, 4)


In [15]:
# Загружаем нужные библиотеки для обучения
import sys
import time

import pandas as pd 
import numpy as np
import os
import utils
from multiprocessing import Pool
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

# модели
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
from scipy.stats import gmean

# для сохранения на диск обученных моделек и матриц преобразований
from sklearn.externals import joblib
import pickle

# кусочек для поиск лучшего набора параметров для моделей (если есть мноого времени) 
from sklearn.grid_search import GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2

# для работы с sparse матрицами
import scipy.sparse as sp


In [16]:
# Загружаем (если надо) подготовленные данные с диска и убиваем строчки с пустыми данными.
#df = pd.read_csv('prep_dataset.csv', encoding='utf-8')
#print(df.shape)
#df.dropna(inplace=True)
#print(df.shape)
#df.isnull().any()
#tf = pd.read_csv('prep_test.csv', encoding='utf-8')
#print(tf.shape)
#tf.isnull().any()

In [17]:
# берем вектор результатов
y = df['category_id'].as_matrix()

In [18]:
# пропускаем GridSearchCV и прочие радости
# берем преобразованный титульник и описание, для каждого отдельно строим матрицу преобразования CountVectorizer+Tfidf
# для каждой берем свой kbest
# далее объеденяем полученные таблицы и наши нормированные колонки с ценой
# обучаем SGD с базовыми настройками
start_time = time.time()
train_output=[]
preproc_tit = 'preproc2_tit'
kbest_tit = 150000
preproc_des = 'preproc1_des'
kbest_des = 500000
price_f = ['price_1_log_nrm', 'price_10_log_nrm', 'price_20_log_nrm', 'price_30_log_nrm', 'price_40_log_nrm', 'price_45_log_nrm', 'price_50_log_nrm', 'price_55_log_nrm', 'price_60_log_nrm', 'price_70_log_nrm', 'price_80_log_nrm', 'price_90_log_nrm', 'price_99_log_nrm', 'price_100_log_nrm']
n_gram = 2
name = 'SGD'

x_tit = df[preproc_tit].as_matrix()
x_des = df[preproc_des].as_matrix()
x_prc = sp.csr_matrix(df[price_f])

tf_tit_transformer = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1,n_gram))),
    ('tfidf', TfidfTransformer(use_idf=True))])
x_tf_tit = tf_tit_transformer.fit_transform(x_tit)

tf_des_transformer = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1,n_gram))),
    ('tfidf', TfidfTransformer(use_idf=True))])
x_tf_des = tf_des_transformer.fit_transform(x_des)

ch2_tit = SelectKBest(chi2, k=kbest_tit)
x_tf_tit_select = ch2_tit.fit_transform(x_tf_tit, y)

ch2_des = SelectKBest(chi2, k=kbest_des)
x_tf_des_select = ch2_des.fit_transform(x_tf_des, y)

x_tf0 = sp.hstack([x_tf_des_select, x_tf_tit_select], format='csr')
x_tf = sp.hstack([x_tf0, x_prc], format='csr')
        
sgd = SGDClassifier(loss='modified_huber',alpha=1e-5,n_iter=700,random_state=666,n_jobs=-1)

skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
accs, fold = [], 0
for trn_idx, val_idx in skf.split(x_tf, y):
    x_trn, x_val = x_tf[trn_idx], x_tf[val_idx]
    y_trn, y_val = y[trn_idx], y[val_idx]
    
    sgd.fit(x_trn, y_trn)

    y_prd = sgd.predict_proba(x_val)
    accs.append(accuracy_score(y_val, np.argmax(y_prd, axis=1)))
stop_time=time.time()
print(f'preproc: {preproc_tit} + {preproc_des}, n_gram:{n_gram}, clf:{name}, mean_acc:{np.mean(accs):0.4f}, std:{np.std(accs):0.4f}, time: {stop_time-start_time:0.4f}')
train_output.append(f'preproc: {preproc_tit} + {preproc_des}, n_gram:{n_gram}, clf:{name}, mean_acc:{np.mean(accs):0.4f}, std:{np.std(accs):0.4f}, time: {stop_time-start_time:0.4f}')

preproc: preproc2_tit + preproc1_des, n_gram:2, clf:SGD, mean_acc:0.8990, std:0.0006, time: 18197.0511


In [19]:
result = sgd.score(x_tf, y)
print(result)

0.945518292999


In [21]:
### save transformer to file 
pickle.dump(tf_tit_transformer, open("tf_tit_transformer_preproc2.pickle", "wb"))
pickle.dump(tf_des_transformer, open("tf_des_transformer_preproc1.pickle", "wb"))
pickle.dump(ch2_tit, open("ch2_tit_150000.pickle", "wb"))
pickle.dump(ch2_des, open("ch2_des_500000.pickle", "wb"))

# save the model to disk
model_filename = 'preproc2_tit_preproc1_des_ngram_1_2_tfidf_sgd.sav'
joblib.dump(sgd, model_filename)

# 53M 337M 11M 67M 268M соответственно

['preproc2_tit_preproc1_des_ngram_1_2_tfidf_sgd.sav']

In [22]:
### load && test:

loaded_tf_tit_transformer = pickle.load(open("tf_tit_transformer_preproc2.pickle", "rb"))
loaded_tf_des_transformer = pickle.load(open("tf_des_transformer_preproc1.pickle", "rb"))
loaded_ch2_tit = pickle.load(open("ch2_tit_150000.pickle", "rb"))
loaded_ch2_des = pickle.load(open("ch2_des_500000.pickle", "rb"))

# load the model from disk
loaded_model = joblib.load(model_filename)
result = loaded_model.score(x_tf, y)
print(result)

# full test

loaded_x_tf_tit = loaded_tf_tit_transformer.transform(x_tit)
loaded_x_tf_des = loaded_tf_des_transformer.transform(x_des)

loaded_x_tf_tit_select = loaded_ch2_tit.transform(loaded_x_tf_tit)
loaded_x_tf_des_select = loaded_ch2_des.transform(loaded_x_tf_des)

loaded_x_tf0 = sp.hstack([loaded_x_tf_des_select, loaded_x_tf_tit_select], format='csr')
loaded_x_tf = sp.hstack([loaded_x_tf0, x_prc], format='csr')

result = loaded_model.score(loaded_x_tf, y)
print(result)

0.945518292999
0.945518292999


In [23]:
# 0.89659
tx_tit = tf[preproc_tit].as_matrix()
tx_des = tf[preproc_des].as_matrix()
tx_prc = sp.csr_matrix(tf[price_f])

tx_tf_tit = tf_tit_transformer.transform(tx_tit)
tx_tf_des = tf_des_transformer.transform(tx_des)

tx_tf_tit_select = ch2_tit.transform(tx_tf_tit)
tx_tf_des_select = ch2_des.transform(tx_tf_des)


tx_tf0 = sp.hstack([tx_tf_des_select, tx_tf_tit_select], format='csr')
tx_tf = sp.hstack([tx_tf0, tx_prc], format='csr')

ty_prd = sgd.predict_proba(tx_tf)

print(np.argmax(ty_prd, axis=1))

ty_ans = np.argmax(ty_prd, axis=1)
data = pd.DataFrame()
data['item_id'] = tf['item_id']
data['category_id'] = ty_ans.tolist()
print(data.head())
data.to_csv('test_ans.csv', index=False, encoding='utf-8')

[22  0 15 ...,  7 18 29]
   item_id  category_id
0   489517           22
1   489518            0
2   489519           15
3   489520           33
4   489521           13
